Modelado de los datos para las funciones de Machine Learning def**recomendacion_juego**

Procedemos a importar librerías

In [12]:
import pandas as pd 
import numpy as np 
import scipy as sp 
from sklearn.metrics.pairwise import cosine_similarity
import fastparquet as fp 
import pyarrow as pa 
import pyarrow.parquet as pq 
from scipy.sparse import csr_matrix

In [4]:
model = pd.read_parquet('Archivos/model.parquet')
model.head(2)

,item_id,app_name,genres
0,761140,Lost Summoner Kitty,Action
1,761140,Lost Summoner Kitty,Casual


In [5]:
#creamos las dummies
dummies = pd.get_dummies(model,columns=['genres'], prefix='')
dummies = dummies.groupby(['item_id', 'app_name']).sum().reset_index()

dummies.head(5)

,item_id,app_name,_Accounting,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,_Design &amp; Illustration,_Early Access,...,_Photo Editing,_RPG,_Racing,_Simulation,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,10,Counter-Strike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50,Half-Life: Opposing Force,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
dummies

,item_id,app_name,_Accounting,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,_Design &amp; Illustration,_Early Access,...,_Photo Editing,_RPG,_Racing,_Simulation,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,10,Counter-Strike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50,Half-Life: Opposing Force,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28844,2028055,Tom Clancy's Ghost Recon Future Soldier - Seas...,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28845,2028056,Worms Revolution Season Pass,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
28846,2028062,Call of Duty®: Black Ops II Season Pass,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28847,2028103,Assassin’s Creed® III Season Pass,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Calculamos la similitud de coseno

In [7]:
similitud = cosine_similarity(dummies.iloc[:,3:])


In [8]:
similitud.shape

(28849, 28849)

In [9]:
similitud

array([[1.        , 1.        , 1.        , ..., 1.        , 0.70710678,
        1.        ],
       [1.        , 1.        , 1.        , ..., 1.        , 0.70710678,
        1.        ],
       [1.        , 1.        , 1.        , ..., 1.        , 0.70710678,
        1.        ],
       ...,
       [1.        , 1.        , 1.        , ..., 1.        , 0.70710678,
        1.        ],
       [0.70710678, 0.70710678, 0.70710678, ..., 0.70710678, 1.        ,
        0.70710678],
       [1.        , 1.        , 1.        , ..., 1.        , 0.70710678,
        1.        ]])

In [10]:
def recomendacion_juego(id):
    id = int(id)

    juego = dummies[dummies['item_id'] == id]

    if juego.empty:
        return 'El juego con el id especificado no existe en la base de datos'
    
    similarity = similitud[dummies[dummies['item_id'] == id].index[0]]
    indices = similarity.argsort()[::-1][1:6]

    recomendacion = dummies.iloc[indices]['app_name']

    return recomendacion


In [12]:
recomendacion_juego(50)

13193                                        Legend (1994)
13340                                      Pang Adventures
13271    Kung Fu Panda: Jombie Monkey, Jombie Shifu, Jo...
13270                          Kung Fu Panda: Spirit Realm
13269                         Kung Fu Panda: Panda Village
Name: app_name, dtype: object

dividiremos el Df para poder montar el modelo en render

In [20]:
#calculamos la mitad 
dummies_mitad = len(dummies) // 10

#selleccionamos la mitad superior 
model_render = dummies.iloc[:dummies_mitad]

In [21]:
model_render.shape

(2884, 24)

In [21]:
#almacenamos el df para render
model_render.to_parquet('Archivos/model_render.parquet')

NameError: name 'model_render' is not defined

In [22]:
model_render= pd.read_parquet('Archivos/model_render.parquet')

In [25]:
"""
    Ingresando el id de producto, deberíamos 
    recibir una lista con 5 juegos recomendados similares al ingresado.
"""

def recomendacion_games(item_id:int):
    juego= model_render[model_render['item_id'] == item_id]

    if juego.empty:
        return("El juego ' {item_id} ' no posee registros en la base de datos")

    #obtenemos el indice del juego dado
    indice = juego.index[0]

    #se toma una muestra aleatoria del Df 
    muestra = 2000 #definimos la muestra
    df_muestra = model_render.sample(n= muestra, random_state=42)

    #calculamos la similitud del contenido para el juego dado y la muestra
    sim_score= cosine_similarity([model_render.iloc[indice, 3:]], df_muestra.iloc[:,3:])

    #obtenemos las puntuacines de similitu del juego dado con otros juegos
    sim_score = sim_score[0]
    #ordenamos los juegos por similitud en orden descendente
    sim_juegos =[(i, sim_score[i]) for i in range(len(sim_score)) if i != indice]
    sim_juegos = sorted(sim_juegos, key=lambda x: x[1], reverse=True)

    #obtenemos los 5 juegos similares
    sim_gam_ind = [i[0] for i in sim_juegos[:5]]

    #lista de los juegos similares
    sim_game_name= df_muestra['app_name'].iloc[sim_gam_ind].tolist()

    return { 'similar_games': sim_game_name}


In [27]:
recomendacion_games(50)

{'similar_games': ['Front Mission Evolved: Wanzer Pack 2',
  'Gotham City Impostors Free to Play: Personality Pack',
  'Hitman: Absolution: Public Enemy Disguise',
  'Shadowgrounds Survivor',
  'Gotham City Impostors Free to Play: Gadget Pack - Professional']}